In [38]:
import pandas as pd
import os

In [76]:
district_path = os.path.join(os.getcwd(),'data_rag/Belgium_districts.xlsx')
district_data = pd.ExcelFile(district_path)

sheets_dict = {}
for sheet_name in district_data.sheet_names:
    globals()[sheet_name] = district_data.parse(sheet_name)
    

In [78]:
provinces_df = district_data.parse('Provinces')

merged_dataframes = []

for sheet_name in district_data.sheet_names:
    if sheet_name != 'Provinces' and sheet_name != 'Hainaut_post2019': # Since our documents are pre2019
        df = district_data.parse(sheet_name)
        
        if 'Province_NL' in df.columns:
            merge_column = 'Province_NL'
        elif 'Province_FR' in df.columns:
            merge_column = 'Province_FR'
        elif 'Province' in df.columns: # For Brussels
            merge_column = 'Province'
        else:
            continue  # Skip sheets without the required columns
        merged_df = pd.merge(df, provinces_df, left_on=merge_column, right_on=merge_column, how='left')
        merged_dataframes.append(merged_df)
combined_df = pd.concat(merged_dataframes, ignore_index=True)
combined_df = combined_df.drop('Municipality_FR', axis=1)

In [80]:
combined_df

,Province_NL,Arrondissement_NL,Arrondissement_FR,Municipality,Province,Province_FR,Province_DE
0,Antwerpen,Antwerpen,Anvers,Aartselaar,Antwerpen,Anvers,Antwerpen
1,Antwerpen,Antwerpen,Anvers,Antwerpen,Antwerpen,Anvers,Antwerpen
2,Antwerpen,Antwerpen,Anvers,Boechout,Antwerpen,Anvers,Antwerpen
3,Antwerpen,Antwerpen,Anvers,Boom,Antwerpen,Anvers,Antwerpen
4,Antwerpen,Antwerpen,Anvers,Borsbeek,Antwerpen,Anvers,Antwerpen
...,...,...,...,...,...,...,...
577,Brussels Hoofdstedelijk Gewest,Brussels,Brussels,Schaarbeek,Brussels-Catpial-Region,Région de Bruxelles-Capitale,Region Brüssel-Hauptstadt
578,Brussels Hoofdstedelijk Gewest,Brussels,Brussels,Ukkel,Brussels-Catpial-Region,Région de Bruxelles-Capitale,Region Brüssel-Hauptstadt
579,Brussels Hoofdstedelijk Gewest,Brussels,Brussels,Watermaal-Bosvoorde,Brussels-Catpial-Region,Région de Bruxelles-Capitale,Region Brüssel-Hauptstadt
580,Brussels Hoofdstedelijk Gewest,Brussels,Brussels,Sint-Lambrechts-Woluwe,Brussels-Catpial-Region,Région de Bruxelles-Capitale,Region Brüssel-Hauptstadt


In [82]:
# combined_df.to_csv(os.path.join(os.getcwd(),'data_rag/Belgium_districts_full.xlsx'))

In [84]:
sector_path = os.path.join(os.getcwd(),'data_rag/Statisticaldistricts_from01012023.xlsx')
sector_data = pd.ExcelFile(sector_path)
sector_df = sector_data.parse(sector_data.sheet_names[0])

merged_df = combined_df.merge(sector_df, how='right', left_on='Municipality', right_on='Commune_NL')
merged_df = merged_df.combine_first(combined_df.merge(sector_df, how='right', left_on='Municipality', right_on='Commune_FR'))
merged_df = merged_df.combine_first(combined_df.merge(sector_df, how='right', left_on='Municipality', right_on='Commune_DE'))

In [85]:
merged_df[merged_df['Province'] == 'Oost-Vlaanderen']['Municipality'].unique()

array(['Aalst', 'Denderleeuw', 'Geraardsbergen', 'Haaltert', 'Herzele',
       'Lede', 'Ninove', 'Sint-Lievens-Houtem', 'Zottegem', 'Erpe-Mere',
       'Berlare', 'Buggenhout', 'Dendermonde', 'Hamme', 'Laarne',
       'Lebbeke', 'Waasmunster', 'Wetteren', 'Wichelen', 'Zele',
       'Assenede', 'Eeklo', 'Kaprijke', 'Maldegem', 'Sint-Laureins',
       'Zelzate', 'Aalter', 'Deinze', 'De Pinte', 'Destelbergen',
       'Evergem', 'Gavere', 'Gent', 'Lochristi', 'Lievegem', 'Melle',
       'Merelbeke', 'Moerbeke', 'Nazareth', 'Oosterzele',
       'Sint-Martens-Latem', 'Wachtebeke', 'Zulte', 'Kruisem',
       'Oudenaarde', 'Ronse', 'Brakel', 'Kluisbergen', 'Wortegem-Petegem',
       'Horebeke', 'Lierde', 'Maarkedal', 'Zwalm', 'Beveren', 'Kruibeke',
       'Lokeren', 'Sint-Gillis-Waas', 'Sint-Niklaas', 'Stekene', 'Temse'],
      dtype=object)

In [98]:
merged_df[merged_df['Province'].isna()].Commune_NL.unique()

array([], dtype=object)

In [96]:
print(len(sector_df), len(merged_df)) # The sector data is from 2023. We consider pre-2019 Hainaut commune structures, which may have resulted in .
print(len(merged_df['Municipality'].unique()))
print(len(combined_df['Province'].unique())) 

19795 19795
581
11


In [118]:
merged_df.to_csv(os.path.join(os.getcwd(),'data_rag/BE_location_full.csv'))

In [115]:
merged_df.Province.unique()

array(['Antwerpen', 'Brussels-Catpial-Region', 'Vlaams-Brabant',
       'Brabant wallon', 'West Flanders', 'Oost-Vlaanderen', 'Hainaut',
       'Liège', 'Limburg', 'Luxembourg', 'Namur'], dtype=object)

In [123]:
merged_df.head()

,Province_NL,Arrondissement_NL,Arrondissement_FR,Municipality,Province,Province_FR,Province_DE,C_NIS5,Commune_NL,Commune_FR,Commune_DE,StatistischeSector_code_Secteurstatistique,Sector_NL,Sector_FR,Sector_DE,C_NIS6
0,Antwerpen,Antwerpen,Anvers,Aartselaar,Antwerpen,Anvers,Antwerpen,11001,Aartselaar,Aartselaar,Aartselaar,11001A00-,AARTSELAAR-CENTRUM,AARTSELAAR-CENTRUM,AARTSELAAR-CENTRUM,11001A
1,Antwerpen,Antwerpen,Anvers,Aartselaar,Antwerpen,Anvers,Antwerpen,11001,Aartselaar,Aartselaar,Aartselaar,11001A01-,DE LEEUWERIK,DE LEEUWERIK,DE LEEUWERIK,11001A
2,Antwerpen,Antwerpen,Anvers,Aartselaar,Antwerpen,Anvers,Antwerpen,11001,Aartselaar,Aartselaar,Aartselaar,11001A020,BUERSTEDE,BUERSTEDE,BUERSTEDE,11001A
3,Antwerpen,Antwerpen,Anvers,Aartselaar,Antwerpen,Anvers,Antwerpen,11001,Aartselaar,Aartselaar,Aartselaar,11001A030,YSSELAAR,YSSELAAR,YSSELAAR,11001A
4,Antwerpen,Antwerpen,Anvers,Aartselaar,Antwerpen,Anvers,Antwerpen,11001,Aartselaar,Aartselaar,Aartselaar,11001A042,KLEINE GRIPPE,KLEINE GRIPPE,KLEINE GRIPPE,11001A
